In [1]:
import random
import time
import torch
import torch.nn as nn
from torch.nn import functional as f
from pathlib import Path
import numpy as np
import torchvision
from torch.utils.data import DataLoader, Dataset
import os
from torchsummary import torchsummary
from PIL import Image

In [10]:
ORIGINAL_IMAGE = 'E:/datasets/NDI_images/Integreted/Observed'
TARGET_IMAGE = 'E:/datasets/NDI_images/Integreted/Calculated'
POSITIVE_RATIO = 0.1
BATCH_SIZE = 8

In [ ]:
ORIGINAL_IMAGE = '/import/mqhome/duanct/dataSets/NDI_images/20220725/20220725/Observed_Crop_200x200pix'
TARGET_IMAGE = '/import/mqhome/duanct/dataSets/NDI_images/20220725/20220725/Calculated_200x200/grayscale'
POSITIVE_RATIO = 0.1
BATCH_SIZE = 8

## CosineEmbeddingLoss

In [4]:
class NDI_dataset(Dataset):
    def __init__(self, positive_ratio=0.5, times=10):
        original_images = list(Path(ORIGINAL_IMAGE).glob('*.jpg'))
        target_images = list(Path(TARGET_IMAGE).glob('*.jpg'))
        self.origins, self.targets, self.label = [], [], []
        for original_image in original_images:
            self.origins.append(torchvision.io.read_image(str(original_image)))
            self.targets.append(torchvision.io.read_image(str(Path.joinpath(Path(TARGET_IMAGE), original_image.name.split('_')[0] + '.jpg'))))
            self.label.append(torch.tensor(1))
        for _ in range(times):
            for original_image in original_images:
                self.origins.append(torchvision.io.read_image(str(original_image)))
                self.targets.append(torchvision.io.read_image(str(np.random.choice(target_images))))
                self.label.append(torch.tensor(-1))

    def __getitem__(self, idx):
        return self.origins[idx], self.targets[idx], self.label[idx]

    def __len__(self):
        return len(self.origins)

In [5]:
train_iter = DataLoader(NDI_dataset(POSITIVE_RATIO), BATCH_SIZE, shuffle=True)

In [6]:
model = torchvision.models.resnet18(pretrained=False)

In [7]:
model.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
model.fc = nn.Linear(512, 256)

In [8]:
torchsummary.summary(model, input_size=(1, 200, 200), device='cpu')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 100, 100]           3,136
       BatchNorm2d-2         [-1, 64, 100, 100]             128
              ReLU-3         [-1, 64, 100, 100]               0
         MaxPool2d-4           [-1, 64, 50, 50]               0
            Conv2d-5           [-1, 64, 50, 50]          36,864
       BatchNorm2d-6           [-1, 64, 50, 50]             128
              ReLU-7           [-1, 64, 50, 50]               0
            Conv2d-8           [-1, 64, 50, 50]          36,864
       BatchNorm2d-9           [-1, 64, 50, 50]             128
             ReLU-10           [-1, 64, 50, 50]               0
       BasicBlock-11           [-1, 64, 50, 50]               0
           Conv2d-12           [-1, 64, 50, 50]          36,864
      BatchNorm2d-13           [-1, 64, 50, 50]             128
             ReLU-14           [-1, 64,

In [9]:
device = torch.device('cuda')
lr = 0.001
EPOCHS = 20

loss = nn.CosineEmbeddingLoss(margin=0.3)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
model.to(device)
train_iter = DataLoader(NDI_dataset(POSITIVE_RATIO), BATCH_SIZE, shuffle=True)
for epoch in range(EPOCHS):
    model.train()
    total_loss = 0
    for origin, target, label in train_iter:
        optimizer.zero_grad()
        origin, target = origin.float(), target.float()
        origin, target, label = origin.to(device), target.to(device), label.to(device)
        origin_feature = model(origin)
        target_feature = model(target)
        l = loss(origin_feature, target_feature, label)
        l.backward()
        total_loss += l
    print(f'Epoch: {epoch + 1}, Ave. Loss: {total_loss / len(train_iter):.4f}')


Epoch: 1, Ave. Loss: 0.5940
Epoch: 2, Ave. Loss: 0.5952
Epoch: 3, Ave. Loss: 0.5944
Epoch: 4, Ave. Loss: 0.5929
Epoch: 5, Ave. Loss: 0.5938
Epoch: 6, Ave. Loss: 0.5953
Epoch: 7, Ave. Loss: 0.5931
Epoch: 8, Ave. Loss: 0.5945
Epoch: 9, Ave. Loss: 0.5908
Epoch: 10, Ave. Loss: 0.5949
Epoch: 11, Ave. Loss: 0.5910
Epoch: 12, Ave. Loss: 0.5936
Epoch: 13, Ave. Loss: 0.5947
Epoch: 14, Ave. Loss: 0.5938
Epoch: 15, Ave. Loss: 0.5946
Epoch: 16, Ave. Loss: 0.5942
Epoch: 17, Ave. Loss: 0.5944
Epoch: 18, Ave. Loss: 0.5940
Epoch: 19, Ave. Loss: 0.5937
Epoch: 20, Ave. Loss: 0.5939


In [10]:
from PIL import Image

model.to('cpu')
original_images = list(Path(ORIGINAL_IMAGE).glob('*.jpg'))
original_image_name = np.random.choice(original_images)
original_image = torchvision.io.read_image(str(original_image_name)).unsqueeze_(0).float()
target_image = torchvision.io.read_image(str(Path.joinpath(Path(TARGET_IMAGE), original_image_name.name.split('_')[0] + '.jpg'))).unsqueeze_(0).float()
print(nn.CosineSimilarity()(model(original_image), model(target_image)))

tensor([0.9949], grad_fn=<DivBackward0>)


In [11]:
model.to('cpu')
original_images = list(Path(ORIGINAL_IMAGE).glob('*.jpg'))
target_images = list(Path(TARGET_IMAGE).glob('*.jpg'))
original_image_name = np.random.choice(original_images)
target_image_name = np.random.choice(target_images)
original_image = torchvision.io.read_image(str(original_image_name)).unsqueeze_(0).float()
target_image = torchvision.io.read_image(str(target_image_name)).unsqueeze_(0).float()
print(nn.CosineSimilarity()(model(original_image), model(target_image)))

tensor([0.9946], grad_fn=<DivBackward0>)


In [129]:
dimensions = 512
a = torch.rand((1, dimensions))
b = torch.rand((1, dimensions))
nn.CosineSimilarity()(a, b)

tensor([0.7585])

## Binary Classification

In [7]:
a = torch.rand((8, 512))
b = torch.rand((8, 512))
torch.cat([a, b], dim=).size()

torch.Size([16, 512])

In [11]:
class ClassificationModel(nn.Module):
    def __init__(self):
        super(ClassificationModel, self).__init__()
        self.backbone = torchvision.models.resnet18(pretrained=False)
        self.backbone.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.backbone.fc = nn.Linear(512, 256)
        self.classifier = nn.Linear(512, 2)
    def forward(self, origin, target):
        origin_feature = f.relu(self.backbone(origin))
        target_feature = f.relu(self.backbone(target))
        return self.classifier(torch.cat([origin_feature, target_feature], dim=-1))

In [12]:
class NDI_dataset_classifier(Dataset):
    def __init__(self, positive_ratio=0.5, times=10):
        original_images = list(Path(ORIGINAL_IMAGE).glob('*.jpg'))
        target_images = list(Path(TARGET_IMAGE).glob('*.jpg'))
        self.origins, self.targets, self.label = [], [], []
        for original_image in original_images:
            self.origins.append(torchvision.io.read_image(str(original_image)))
            self.targets.append(torchvision.io.read_image(str(Path.joinpath(Path(TARGET_IMAGE), original_image.name.split('_')[0] + '.jpg'))))
            self.label.append(torch.tensor(1, dtype=torch.long))
        for _ in range(times):
            for original_image in original_images:
                self.origins.append(torchvision.io.read_image(str(original_image)))
                self.targets.append(torchvision.io.read_image(str(np.random.choice(target_images))))
                self.label.append(torch.tensor(0, dtype=torch.long))

    def __getitem__(self, idx):
        return self.origins[idx], self.targets[idx], self.label[idx]

    def __len__(self):
        return len(self.origins)

In [16]:
model = ClassificationModel()
torchsummary.summary(model, input_size=[(1, 200, 200), (1, 200, 200)], device='cpu')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 100, 100]           3,136
       BatchNorm2d-2         [-1, 64, 100, 100]             128
              ReLU-3         [-1, 64, 100, 100]               0
         MaxPool2d-4           [-1, 64, 50, 50]               0
            Conv2d-5           [-1, 64, 50, 50]          36,864
       BatchNorm2d-6           [-1, 64, 50, 50]             128
              ReLU-7           [-1, 64, 50, 50]               0
            Conv2d-8           [-1, 64, 50, 50]          36,864
       BatchNorm2d-9           [-1, 64, 50, 50]             128
             ReLU-10           [-1, 64, 50, 50]               0
       BasicBlock-11           [-1, 64, 50, 50]               0
           Conv2d-12           [-1, 64, 50, 50]          36,864
      BatchNorm2d-13           [-1, 64, 50, 50]             128
             ReLU-14           [-1, 64,

In [ ]:
device = torch.device('cuda')
lr = 0.001
EPOCHS = 20

loss = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
model.to(device)
train_iter = DataLoader(NDI_dataset(POSITIVE_RATIO), BATCH_SIZE, shuffle=True)
for epoch in range(EPOCHS):
    model.train()
    total_loss = 0
    for origin, target, label in train_iter:
        optimizer.zero_grad()
        origin, target = origin.float(), target.float()
        origin, target, label = origin.to(device), target.to(device), label.to(device)
        origin_feature = model(origin)
        target_feature = model(target)
        l = loss(origin_feature, target_feature, label)
        l.backward()
        total_loss += l
    print(f'Epoch: {epoch + 1}, Ave. Loss: {total_loss / len(train_iter):.4f}')

In [17]:
x = torch.rand((1, 2))

In [20]:
torch.argmax(x).type(x.dtype).sum()

tensor(0.)

In [23]:
torch.rand(1)

tensor([0.6341])

In [28]:
mnist_dataset = torchvision.datasets.FashionMNIST(root='../data', train=True, transform=torchvision.transforms.Compose([torchvision.transforms.ToTensor()]), download=True)
mnist_loader = torch.utils.data.DataLoader(mnist_dataset, batch_size=32, shuffle=True)
for x, y in mnist_loader:
    print(x.shape)
    print(y.shape)
    break

torch.Size([32, 1, 28, 28])
torch.Size([32])


In [33]:
torch.max(x[0], dim=2)

torch.return_types.max(
values=tensor([[0.7490, 0.9333, 0.8863, 0.8824, 0.8784, 0.8196, 0.8078, 0.8510, 0.8941,
         0.9647, 0.9176, 1.0000, 0.8588, 0.9216, 0.9294, 0.9882, 0.9882, 0.9882,
         0.9843, 0.9765, 0.9451, 0.9843, 0.9843, 0.9412, 0.8627, 0.9020, 0.8431,
         0.6902]]),
indices=tensor([[13, 15, 10, 16,  6, 18, 18,  7,  7, 19, 19, 19, 20,  9,  9,  6, 18, 18,
         18, 18, 18, 21, 21,  8, 21, 19, 19, 22]]))

In [3]:
class NDIDatasetContrastiveLearning(Dataset):
    def __init__(self):
        super(NDIDatasetContrastiveLearning, self).__init__()
        original_images = list(Path(ORIGINAL_IMAGE).glob('*.jpg'))
        origins, targets, label = [], [], []
        to_tensor_func = torchvision.transforms.ToTensor()
        for original_image in original_images:
            origins.append(to_tensor_func(Image.open(str(original_image))))
            targets.append(to_tensor_func(Image.open(str(Path.joinpath(Path(TARGET_IMAGE), original_image.name.split('_')[0] + '.jpg')))))
            label.append(int(original_image.name.split('_')[0]) - 1)
        self.origins, self.targets, self.label = origins, targets, label
        # random_index = np.random.permutation(len(origins))
        # self.origins, self.targets, self.label = [], [], []
        # for index in random_index:
        #     self.origins.append(origins[index])
        #     self.targets.append(targets[index])
        #     self.label.append(index)

    def __getitem__(self, idx):
        return self.origins[idx], self.targets[idx], self.label[idx]

    def __len__(self):
        return len(self.origins)

In [ ]:
class NDIDatasetContrastiveLearningDataAug(Dataset):
    def __init__(self):
        super(NDIDatasetContrastiveLearningDataAug, self).__init__()
        original_images = list(Path(ORIGINAL_IMAGE).glob('*.jpg'))
        origins, targets, label = [], [], []
        self.to_tensor_func = torchvision.transforms.ToTensor()
        for original_image in original_images:
            original_img = Image.open(str(original_image))
            target_img = Image.open(str(Path.joinpath(Path(TARGET_IMAGE), original_image.name.split('_')[0] + '.jpg')))
            original_img, target_img = self.transform(original_img, target_img)
            origins.append(original_img)
            targets.append(target_img)
            label.append(int(original_image.name.split('_')[0]) - 1)
        self.origins, self.targets, self.label = origins, targets, label
        # random_index = np.random.permutation(len(origins))
        # self.origins, self.targets, self.label = [], [], []
        # for index in random_index:
        #     self.origins.append(origins[index])
        #     self.targets.append(targets[index])
        #     self.label.append(index)

    def __getitem__(self, idx):
        return self.origins[idx], self.targets[idx], self.label[idx]

    def __len__(self):
        return len(self.origins)

    def transform(self, img1, img2):
        random_number = random.random()
        if random_number < 0.33:
            if random.random() < 0.5:
                img1 = torchvision.transforms.functional.hflip(img1)
                img2 = torchvision.transforms.functional.hflip(img2)
            if random.random() < 0.5:
                img1 = torchvision.transforms.functional.vflip(img1)
                img2 = torchvision.transforms.functional.vflip(img2)
        elif random_number < 0.7:
            random_angle = torchvision.transforms.RandomRotation.get_params([-90, 90])
            img1 = torchvision.transforms.functional.rotate(img1, random_angle)
            img2 = torchvision.transforms.functional.rotate(img2, random_angle)
        return self.to_tensor_func(img1), self.to_tensor_func(img2)

In [11]:
from moco_model import MoCo


model = MoCo(torchvision.models.resnet18, dim=128, K=40)
device = torch.device('cuda:0')
criterion = nn.CrossEntropyLoss().cuda(device)
optimizer = torch.optim.SGD(model.parameters(), lr=0.005, momentum=0.9, weight_decay=1e-4)

In [58]:
dataset = NDIDatasetContrastiveLearning()
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [40, 14])
train_iter = DataLoader(train_dataset, batch_size=8, shuffle=True, drop_last=True)
val_iter = DataLoader(val_dataset, batch_size=len(val_dataset))
to_tensor_func = torchvision.transforms.ToTensor()
target_tensor = []
for i in range(1, 55):
    target_tensor.append(to_tensor_func(Image.open(str(Path.joinpath(Path(TARGET_IMAGE), f'{i}.jpg')))).unsqueeze(0))
target_tensor = torch.cat(target_tensor, dim=0)
target_tensor = target_tensor.cuda(device)
model.cuda(device)
for origin, _, label in val_iter:
    origin, label = origin.cuda(device), label.cuda(device)
    q = model.encoder_q(origin)
    q = f.normalize(q, dim=1)
    k = model.encoder_k(target_tensor)
    k = f.normalize(k, dim=1)
    logits = torch.mm(q, k.T)
    preds = torch.argmax(logits, dim=1)
    print(torch.sum(preds.type_as(label) == label))

tensor(0, device='cuda:0')


In [75]:
l_neg = torch.einsum('nc,ck->nk', [q, k.T])
l_neg

tensor([[0.9566, 0.9737, 0.9622, 0.9789, 0.9772, 0.9746, 0.9748, 0.9764, 0.9692,
         0.9736, 0.9824, 0.9508, 0.9770, 0.9738, 0.9807, 0.8780, 0.9732, 0.9759,
         0.9735, 0.9712, 0.9789, 0.9776, 0.9470, 0.9700, 0.9749, 0.9749, 0.9775,
         0.8600, 0.8569, 0.9710, 0.9315, 0.9395, 0.9240, 0.9727, 0.9443, 0.9284,
         0.9813, 0.9683, 0.9805, 0.9707, 0.8684, 0.9756, 0.9765, 0.8677, 0.8620,
         0.9690, 0.9724, 0.9831, 0.9756, 0.9755, 0.9732, 0.8786, 0.8786, 0.9761],
        [0.8980, 0.9588, 0.9742, 0.9485, 0.9552, 0.9592, 0.9695, 0.9412, 0.9699,
         0.9617, 0.9651, 0.8908, 0.9637, 0.9763, 0.9623, 0.9404, 0.9670, 0.9623,
         0.9509, 0.9670, 0.9708, 0.9638, 0.8875, 0.9633, 0.9731, 0.9731, 0.9593,
         0.9233, 0.9180, 0.9660, 0.8786, 0.8786, 0.9607, 0.9524, 0.8836, 0.9681,
         0.9570, 0.9758, 0.9712, 0.9782, 0.9272, 0.9494, 0.9666, 0.9345, 0.9131,
         0.9709, 0.9676, 0.9566, 0.9366, 0.9362, 0.9623, 0.9421, 0.9421, 0.9698],
        [0.8853, 0.9572, 0

In [81]:
def train_moco(net, criterion, optimizer, epochs, device):
    dataset = NDIDatasetContrastiveLearning()
    train_dataset, val_dataset = torch.utils.data.random_split(dataset, [40, 14])
    train_iter = DataLoader(train_dataset, batch_size=8, shuffle=True, drop_last=True)
    val_iter = DataLoader(val_dataset, batch_size=len(val_dataset))
    to_tensor_func = torchvision.transforms.ToTensor()
    target_tensor = []
    for i in range(1, 55):
        target_tensor.append(to_tensor_func(Image.open(str(Path.joinpath(Path(TARGET_IMAGE), f'{i}.jpg')))).unsqueeze(0))
    target_tensor = torch.cat(target_tensor, dim=0)
    target_tensor = target_tensor.cuda(device)
    for epoch in range(epochs):
        net.cuda(device)
        total_loss = 0
        training_correct = 0
        training_size = 0
        for origin, target, label in train_iter:
            net.train()
            origin, target, label = origin.cuda(device), target.cuda(device), label.cuda(device)
            output, labels = net(origin, target)
            loss = criterion(output, labels)
            total_loss += loss.item()
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            net.eval()
            training_correct += cal_accuracy(image_pair_matching(net, origin, target_tensor), label)
            training_size += origin.shape[0]
        net.eval()
        for origin, _, label in val_iter:
            origin, label = origin.cuda(device), label.cuda(device)
            val_correct = cal_accuracy(image_pair_matching(net, origin, target_tensor), label)
        val_acc = val_correct / origin.shape[0]
        print(f'Epoch {epoch + 1}, Loss {total_loss / len(train_iter)}, Train_acc {training_correct / training_size}, Val_acc {val_acc}')

In [ ]:
# 把K设置成48还算成功，在queue不大的情况下

train_moco(model, criterion, optimizer, 50, device)

Epoch 1, Loss 1.7299450635910034, Train_acc 0.7, Val_acc 0.7857142857142857
Epoch 2, Loss 1.6755859851837158, Train_acc 0.7, Val_acc 0.7857142857142857
Epoch 3, Loss 1.4713821172714234, Train_acc 0.625, Val_acc 0.7857142857142857
Epoch 4, Loss 1.4476009130477905, Train_acc 0.85, Val_acc 0.7857142857142857
Epoch 5, Loss 1.4682427644729614, Train_acc 0.9, Val_acc 0.7857142857142857
Epoch 6, Loss 1.4163634061813355, Train_acc 0.825, Val_acc 0.7142857142857143
Epoch 7, Loss 1.3091506481170654, Train_acc 0.95, Val_acc 0.7857142857142857
Epoch 8, Loss 1.2512558221817016, Train_acc 0.95, Val_acc 0.7857142857142857
Epoch 9, Loss 1.224940299987793, Train_acc 0.95, Val_acc 0.7857142857142857
Epoch 10, Loss 1.2253174304962158, Train_acc 0.95, Val_acc 0.7857142857142857
Epoch 11, Loss 1.245502781867981, Train_acc 0.95, Val_acc 0.7857142857142857
Epoch 12, Loss 1.213822364807129, Train_acc 0.95, Val_acc 0.7857142857142857
Epoch 13, Loss 1.230686330795288, Train_acc 0.95, Val_acc 0.7857142857142857


In [95]:
# 尝试把K设置成56

model = MoCo(torchvision.models.resnet18, dim=128, K=56)
device = torch.device('cuda:0')
criterion = nn.CrossEntropyLoss().cuda(device)
optimizer = torch.optim.SGD(model.parameters(), lr=0.005, momentum=0.9, weight_decay=1e-4)
train_moco(model, criterion, optimizer, 50, device)

Epoch 1, Loss 2.673976675757634
Epoch 2, Loss 3.931485931078593
Epoch 3, Loss 3.8541926542917886
Epoch 4, Loss 3.6459306478500366
Epoch 5, Loss 3.407223383585612
Epoch 6, Loss 3.105238596598307
Epoch 7, Loss 3.1296446720759072
Epoch 8, Loss 3.1514963706334433
Epoch 9, Loss 3.053861141204834
Epoch 10, Loss 2.6810205380121865
Epoch 11, Loss 2.705527981122335
Epoch 12, Loss 2.632307688395182
Epoch 13, Loss 2.4753270546595254
Epoch 14, Loss 2.3976933558781943
Epoch 15, Loss 2.333985129992167
Epoch 16, Loss 2.2366109689076743
Epoch 17, Loss 2.1844319701194763
Epoch 18, Loss 2.2525221506754556
Epoch 19, Loss 2.1583401759465537
Epoch 20, Loss 2.0467366774876914
Epoch 21, Loss 2.0968031088511148
Epoch 22, Loss 2.081486225128174
Epoch 23, Loss 2.047618548075358
Epoch 24, Loss 1.9676475127538045
Epoch 25, Loss 1.9295819004376729
Epoch 26, Loss 1.8663320342699687
Epoch 27, Loss 1.8291430870691936
Epoch 28, Loss 1.789828399817149
Epoch 29, Loss 1.7487680315971375
Epoch 30, Loss 1.7118704319000244


In [12]:
def image_pair_matching(net, original_image, matching_image):
    net.eval()
    q = net.encoder_q(original_image)
    q = f.normalize(q, dim=1)
    k = net.encoder_k(matching_image)
    k = f.normalize(k, dim=1)
    logits = torch.einsum('nc,ck->nk', [q, k.T])
    return logits

In [13]:
def cal_accuracy(preds, label):
    return float(torch.sum(torch.argmax(preds, dim=1).type_as(label) == label))

In [93]:
# 取总共大小小于K的queue来试，成功

model.cpu()
original_images = list(Path(ORIGINAL_IMAGE).glob('*.jpg'))
target_images = list(Path(TARGET_IMAGE).glob('*.jpg'))
to_tensor_func = torchvision.transforms.ToTensor()
original_image = np.random.choice(original_images)
original_tensor = to_tensor_func(Image.open(str(original_image))).unsqueeze(0)
target_images_candidates = np.random.choice(target_images, 40)
target_tensor = []
for i in range(20):
    target_tensor.append(to_tensor_func(Image.open(str(target_images_candidates[i]))).unsqueeze(0))
target_tensor.append(to_tensor_func(Image.open(str(Path.joinpath(Path(TARGET_IMAGE), original_image.name.split('_')[0] + '.jpg')))).unsqueeze(0))
for i in range(20, 40):
    target_tensor.append(to_tensor_func(Image.open(str(target_images_candidates[i]))).unsqueeze(0))
target_tensor = torch.cat(target_tensor, dim=0)

In [94]:
image_pair_matching(model, original_tensor, target_tensor)

tensor([[ 0.2281,  0.1505,  0.0759,  0.3091,  0.0944,  0.3091, -0.0131,  0.1535,
          0.0576,  0.1025,  0.0797,  0.1233,  0.1377,  0.0640,  0.1535,  0.0891,
         -0.0098,  0.0383,  0.2141, -0.0383,  0.3091,  0.0969,  0.0739,  0.0576,
          0.1209,  0.1377,  0.0891,  0.1424,  0.0175,  0.1025,  0.1732,  0.0883,
          0.0811,  0.0613,  0.1233, -0.0235,  0.1025,  0.0883,  0.0738,  0.2119,
          0.0383]], grad_fn=<MmBackward>)

In [113]:
# 取所有比对图片

model.cpu()
original_images = list(Path(ORIGINAL_IMAGE).glob('*.jpg'))
target_images = list(Path(TARGET_IMAGE).glob('*.jpg'))
to_tensor_func = torchvision.transforms.ToTensor()
original_image = np.random.choice(original_images)
original_tensor = to_tensor_func(Image.open(str(original_image))).unsqueeze(0)
target_tensor = []
for i in range(1, 55):
    target_tensor.append(to_tensor_func(Image.open(str(Path.joinpath(Path(TARGET_IMAGE), f'{i}.jpg')))).unsqueeze(0))
target_tensor = torch.cat(target_tensor, dim=0)
print(str(original_image.name).split('_')[0])

21


In [114]:
logits = image_pair_matching(model, original_tensor, target_tensor)
print(logits)
print(torch.argmax(logits))

tensor([[ 1.4565e-01,  1.1097e-01,  3.9617e-02,  1.2304e-01,  1.0800e-01,
          1.1116e-01,  1.9075e-01,  1.6977e-01,  7.8928e-02,  2.5626e-01,
          9.6446e-02,  1.6837e-01,  8.6016e-02,  1.0304e-01,  9.4899e-02,
          3.0067e-02,  8.0622e-02,  1.3401e-01,  8.8296e-02, -1.2011e-03,
          4.0938e-01,  5.5814e-02,  1.7832e-01,  6.8754e-02,  3.9356e-01,
          3.9356e-01,  2.4748e-01, -9.8140e-02, -9.8763e-02,  1.2288e-01,
          1.8330e-01,  1.7046e-01,  7.5117e-02,  1.4688e-01,  1.3979e-01,
          1.2716e-04,  1.2593e-01,  7.9922e-02,  3.0441e-01,  3.9980e-02,
          2.0772e-02,  1.7252e-01,  1.3889e-01,  3.5146e-03,  3.6427e-02,
          1.8339e-01,  8.3577e-02,  5.3283e-02,  1.7072e-01,  1.6657e-01,
          1.9643e-01,  5.9414e-02,  5.9414e-02,  1.3753e-01]],
       grad_fn=<MmBackward>)
tensor(20)


In [5]:
dataset = NDIDatasetContrastiveLearning()
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [43, 11])
train_iter = DataLoader(train_dataset, batch_size=8, shuffle=True, drop_last=True)
val_iter = DataLoader(val_dataset, batch_size=1)
for original, target in val_iter:
    print(original.shape)
    print(target.shape)
    break

torch.Size([1, 1, 200, 200])
torch.Size([1, 1, 200, 200])


In [14]:
def train_moco_no_val(net, criterion, optimizer, epochs, device):
    dataset = NDIDatasetContrastiveLearning()
    train_iter = DataLoader(dataset, batch_size=8, shuffle=True, drop_last=True)
    to_tensor_func = torchvision.transforms.ToTensor()
    target_tensor = []
    for i in range(1, 55):
        target_tensor.append(to_tensor_func(Image.open(str(Path.joinpath(Path(TARGET_IMAGE), f'{i}.jpg')))).unsqueeze(0))
    target_tensor = torch.cat(target_tensor, dim=0)
    target_tensor = target_tensor.cuda(device)
    for epoch in range(epochs):
        net.cuda(device)
        total_loss = 0
        training_correct = 0
        training_size = 0
        for origin, target, label in train_iter:
            net.train()
            origin, target, label = origin.cuda(device), target.cuda(device), label.cuda(device)
            output, labels = net(origin, target)
            loss = criterion(output, labels)
            total_loss += loss.item()
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            net.eval()
            training_correct += cal_accuracy(image_pair_matching(net, origin, target_tensor), label)
            training_size += origin.shape[0]
        print(f'Epoch {epoch + 1}, Loss {total_loss / len(train_iter)}, Train_acc {training_correct / training_size}')

In [15]:
start_time = time.time()
train_moco_no_val(model, criterion, optimizer, 50, device)
end_time = time.time()
print(f'Cost {end_time - start_time} secs')

Epoch 1, Loss 2.5786702330660773, Train_acc 0.020833333333333332
Epoch 2, Loss 3.732647697130839, Train_acc 0.020833333333333332
Epoch 3, Loss 3.5257873932520547, Train_acc 0.041666666666666664
Epoch 4, Loss 3.495723605155945, Train_acc 0.020833333333333332
Epoch 5, Loss 3.1676207780838013, Train_acc 0.020833333333333332
Epoch 6, Loss 2.9918357928593955, Train_acc 0.020833333333333332
Epoch 7, Loss 2.993980884552002, Train_acc 0.020833333333333332
Epoch 8, Loss 3.0280898809432983, Train_acc 0.0
Epoch 9, Loss 2.756258209546407, Train_acc 0.020833333333333332
Epoch 10, Loss 2.701042890548706, Train_acc 0.0
Epoch 11, Loss 2.528733491897583, Train_acc 0.10416666666666667
Epoch 12, Loss 2.5128378868103027, Train_acc 0.22916666666666666
Epoch 13, Loss 2.264402707417806, Train_acc 0.4791666666666667
Epoch 14, Loss 2.3837443590164185, Train_acc 0.4166666666666667
Epoch 15, Loss 2.1378223299980164, Train_acc 0.6458333333333334
Epoch 16, Loss 2.1182019114494324, Train_acc 0.7291666666666666
Epoc

In [35]:
train_iter = DataLoader(NDIDatasetContrastiveLearning(), batch_size=8, shuffle=True, drop_last=True)
x1, x2, x3, y1, y2, y3 = [], [], [], [], [], []
for origin, target, label in train_iter:
    x1.append(origin)
    x2.append(target)
    x3.append(label)
for origin, target, label in train_iter:
    y1.append(origin)
    y2.append(target)
    y3.append(label)
x1 = torch.cat(x1, dim=0).squeeze(1)
y1 = torch.cat(y1, dim=0).squeeze(1)
for i in range(x1.shape[0]):
    flag = False
    for j in range(y1.shape[0]):
        if (x1[i] == y1[j]).sum() == 40000:
            print(True)
            flag = True
            break
    if not flag:
        print(False)

True
True
True
True
True
True
True
True
False
True
False
False
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
False
True
True
True
True
False
True
True
True
True
True
True
True
True
True
True
True
True
True
False
True


In [2]:
# rename part

rename_path = Path('E:/datasets/NDI_images/20220917_ElectronDiffraction-20220919T162109Z-001/20220917_ElectronDiffraction/Obs_200x200_jpg')
files =rename_path.glob('*.jpg')
for file in files:
    new_name = str(file.name).split('_')
    new_name[0] = str(int(new_name[0]) + 59)
    new_name = '_'.join(new_name)
    file.rename(str(Path.joinpath(rename_path, new_name)))

In [8]:
rename_path = Path('E:/datasets/NDI_images/20220917_ElectronDiffraction-20220919T162109Z-001/20220917_ElectronDiffraction/Cal_200x200_png')
files = rename_path.glob('*.png')
files = sorted(list(files), key=lambda x: int(str(x.name).split('.')[0]), reverse=True)
for file in files:
    new_name = str(file.name).split('.')
    new_name[0] = str(int(new_name[0]) + 59)
    new_name = '.'.join(new_name)
    file.rename(str(Path.joinpath(rename_path, new_name)))